### Real Time Age Estimator ###

In [1]:
import cv2
import dlib
import face_recognition

detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor("predictor/shape_predictor_5_face_landmarks.dat")

In [ ]:
import os
from keras.models import load_model

os.environ["CUDA_VISIBLE_DEVICES"]="0"


model = load_model('../../dataset/models/state_meh.h5')


In [2]:
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
	# to the format (x, y, w, h) as we would normally do
	# with OpenCV
	x = rect.left()
	y = rect.top()
	w = rect.right() - x
	h = rect.bottom() - y
 
	# return a tuple of (x, y, w, h)
	return (x, y, w, h)

In [6]:
video_capture = cv2.VideoCapture(0)

count = 0

while(True):
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    
    print(frame)
    
    if count == 5:
        count = 0
        continue
    else:
        count += 1
        
    dets = detector(frame, 1)
    
    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)], then draw the face bounding box
        (x, y, w, h) = face_utils.rect_to_bb(rect)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # show the face number
        cv2.putText(image, "Face #{}".format(i + 1), (x - 10, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # loop over the (x, y)-coordinates for the facial landmarks
        # and draw them on the image
        for (x, y) in shape:
            cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
    
    

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

None


RuntimeError: Unsupported image type, must be 8bit gray or RGB image.